In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install kaleido

     |████████████████████████████████| 79.9 MB 184 kB/s             


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go
import glob
import os

In [4]:
def create_dir (direct):
  if not os.path.exists(direct):
    os.mkdir(direct)

In [5]:
ROOT_DIR = "/kaggle/input/ferset" #MT Directory
DATASET_02 = ROOT_DIR + "/02_FER"
DATASET_02_TRAIN = DATASET_02 + "/train"
DATASET_02_TEST = DATASET_02+ "/test"

In [6]:
import PIL

image = PIL.Image.open(DATASET_02_TRAIN + "/surprise/Training_99984132.jpg")
#image to open

width, height = image.size

print(image.mode)
print(width, height)

L
48 48


In [7]:
datagen = ImageDataGenerator()
image_size = (48, 48)
colormode = 'rgb'
test_set = datagen.flow_from_directory(DATASET_02_TEST, target_size=image_size, color_mode =colormode, batch_size=32, shuffle=False)
train_set = datagen.flow_from_directory(DATASET_02_TRAIN, target_size=image_size, color_mode =colormode, batch_size=32)

Found 7178 images belonging to 7 classes.
Found 28709 images belonging to 7 classes.


In [8]:
def create_model_V1():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))
    
    return model


model_1 = create_model_V1()
model_1.summary()

2022-04-10 05:29:45.995780: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               3277056   
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 1

2022-04-10 05:29:46.082142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 05:29:46.083119: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 05:29:46.084550: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-10 05:29:46.085684: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [9]:
def get_data_df(dir):
  # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

  train_df = pd.DataFrame()

  trainset = glob.glob(dir)
  #print(trainset)
  train_df['filename'] = [img.split("/")[-2]+"/"+img.split("/")[-1] for img in trainset]
  train_df['label'] = [img.split("/")[-2] for img in trainset]

  return train_df

In [10]:
test_train = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
test_train

,filename,label
0,surprise/Training_66056468.jpg,surprise
1,surprise/Training_26522394.jpg,surprise
2,surprise/Training_63239185.jpg,surprise
3,surprise/Training_67725241.jpg,surprise
4,surprise/Training_71308969.jpg,surprise
...,...,...
28704,happy/Training_18866880.jpg,happy
28705,happy/Training_16513509.jpg,happy
28706,happy/Training_10834907.jpg,happy
28707,happy/Training_30459679.jpg,happy


In [11]:
total = 0
value_list = []
for item in os.listdir(DATASET_02_TRAIN):
    y = len(os.listdir(DATASET_02_TRAIN+'/'+item))+len(os.listdir(DATASET_02_TEST+'/'+item))
    value_list.append(y)
    total += y
    print(item)
print(total)
print(value_list)
weights = {}
for n in range(0,len(value_list)):
    w = total/(len(value_list)*value_list[n])
    weights[n] = w
    
print(weights)

surprise
fear
angry
neutral
sad
disgust
happy
35887
[4002, 5121, 4953, 6198, 6077, 547, 8989]
{0: 1.2810380524023703, 1: 1.001115853488437, 2: 1.035072539009547, 3: 0.8271562255105334, 4: 0.8436258492207152, 5: 9.372420997649517, 6: 0.5703319930708962}


In [12]:
create_dir('/kaggle/working/Saved_CNN_Models')


In [13]:
from sklearn.model_selection import KFold, StratifiedKFold

train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
Y = train_data[['label']]

#Y = final_train[['label']]
#n = len(final_train)   

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

#Un needed preprocessing aside from rescale
#idg = ImageDataGenerator(rescale=1./255)
create_dir('/kaggle/working/Saved_CNN_Models')
#model name
def get_model_name(k):
    return 'model_'+str(k)+'.h5'
def model_test(fold_count, epoch, model_method, image_size, mode):
    skf = StratifiedKFold(n_splits = fold_count, random_state = 7, shuffle = True)
    train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
      #Incorperate preprocessing in the method 
      #####################################################################
      #aug_train = get_data_df(DATASET_01_ROOT+"/Aug_train/*/*.jpg")
      #aug_train['filename'] = aug_train['filename'].str[7:]
      #final_train = test_train.append(aug_train, sort=False, ignore_index=True)


      #######################################################################
      #idg = ImageDataGenerator(rescale=1./255)
    idg = ImageDataGenerator()
    Y = train_data[['label']]
    n = len(train_data)    
    VALIDATION_ACCURACY = []
    VALIDATION_LOSS = []

      #create a directory
    save_dir = '/kaggle/working/Saved_CNN_Models/'+model_method.__name__+'_'+str(epoch)+'epochs/'
    create_dir(save_dir)
    
    img_dir = DATASET_02_TRAIN

    fold_var = 1

      #n is number of samples, doesn't care about x values for split, just y
    for train_index, val_index in skf.split(np.zeros(n),Y):

        #Index value for fold
        training_data = train_data.iloc[train_index]
        validation_data = train_data.iloc[val_index]
        #generator for train
        #print("hi")
        #print(validation_data.head())
        #Modify the training size, have an input adjustability 
        train_data_generator = idg.flow_from_dataframe(training_data, directory = img_dir,
                                                       target_size=image_size, batch_size=32, color_mode =mode,
                                                       x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #generator for validation
        valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = img_dir,
                                                        target_size=image_size, color_mode =mode,batch_size=32,
                                                        x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #create a method for this
        # CREATE NEW MODEL
        model = model_method()
        # COMPILE NEW MODEL
        model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])

        # CREATE CALLBACKS
        #Unsure about checpoint
        checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+model_method.__name__+"_"+get_model_name(fold_var), 
                    monitor='val_accuracy', verbose=1, 
                    save_best_only=True, mode='max')

        callbacks_list = [checkpoint]
        # There can be other callbacks, but just showing one because it involves the model name
        # This saves the best model
        # FIT THE MODEL
        #Include the weights for classes
        history = model.fit(train_data_generator,
                epochs=epoch,
                callbacks=callbacks_list,
                validation_data=valid_data_generator,
                class_weight = weights,
                verbose = 2)

        #PLOT HISTORY
        #Plots for whichever
        #		:
        #plt.plot(history.history['binary_accuracy'], label='accuracy')
        #plt.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
        #plt.xlabel('Epoch')
        #plt.ylabel('Accuracy')
        #plt.ylim([0.3, 1])
        #plt.legend(loc='lower right')
        #plt.show()
        #frame = {'Accuracy': history.history['binary_accuracy'], 'Val_Accuracy': history.history['val_binary_accuracy']}
        #fig = px.line(frame)
        #fig.show()
        fig = go.Figure()
        fig.add_trace(go.Scatter(y= history.history['accuracy'],
                          mode='lines+markers',
                          name='Training Accuracy'))
        fig.add_trace(go.Scatter(y= history.history['val_accuracy'],
                          mode='lines+markers',
                          name='Validation Accuracy'))
        fig.update_yaxes(range=[0, 1])
        fig.update_layout(
            title="Fold "+ str(fold_var)+ "'s performance",
            title_x=0.5,
            xaxis_title="Epochs",
            yaxis_title="Accuracy",
            autosize=False,
            width=800,
            height=400,
            margin=dict(l=20, r=20, t=40, b=20),)
        fig.show()
        graph_dir = '/kaggle/working/img'
        create_dir(graph_dir)
        graph_dir = graph_dir+'/'+model_method.__name__
        create_dir(graph_dir)
        fig.write_image(graph_dir+'/fold'+str(fold_var)+'.png') 

        #		:
        # LOAD BEST MODEL to evaluate the performance of the model
        model.load_weights(save_dir+model_method.__name__+"_" + "model_"+str(fold_var)+".h5")

        results = model.evaluate(test_set)
        results = dict(zip(model.metrics_names,results))

        VALIDATION_ACCURACY.append(results['accuracy'])
        VALIDATION_LOSS.append(results['loss'])

        tf.keras.backend.clear_session()

        fold_var += 1

In [14]:
model_test(5, 1, create_model_V1, (48,48), 'rgb')

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.


2022-04-10 05:29:59.619222: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-10 05:30:00.904958: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


718/718 - 82s - loss: 5.5856 - accuracy: 0.1970 - val_loss: 2.5223 - val_accuracy: 0.1897

Epoch 00001: val_accuracy improved from -inf to 0.18966, saving model to /kaggle/working/Saved_CNN_Models/create_model_V1_1epochs/create_model_V1_model_1.h5


225/225 [==============================] - 43s 194ms/step - loss: 2.5135 - accuracy: 0.1963
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
718/718 - 22s - loss: 4.8884 - accuracy: 0.1997 - val_loss: 2.3876 - val_accuracy: 0.2158

Epoch 00001: val_accuracy improved from -inf to 0.21578, saving model to /kaggle/working/Saved_CNN_Models/create_model_V1_1epochs/create_model_V1_model_2.h5


225/225 [==============================] - 6s 27ms/step - loss: 2.3900 - accuracy: 0.2283
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
718/718 - 23s - loss: 7.7684 - accuracy: 0.1946 - val_loss: 2.3625 - val_accuracy: 0.1992

Epoch 00001: val_accuracy improved from -inf to 0.19923, saving model to /kaggle/working/Saved_CNN_Models/create_model_V1_1epochs/create_model_V1_model_3.h5


225/225 [==============================] - 6s 25ms/step - loss: 2.3698 - accuracy: 0.2047
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
718/718 - 24s - loss: 6.0666 - accuracy: 0.1924 - val_loss: 2.2841 - val_accuracy: 0.2151

Epoch 00001: val_accuracy improved from -inf to 0.21508, saving model to /kaggle/working/Saved_CNN_Models/create_model_V1_1epochs/create_model_V1_model_4.h5


225/225 [==============================] - 5s 24ms/step - loss: 2.2564 - accuracy: 0.2211
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
718/718 - 23s - loss: 5.9584 - accuracy: 0.1952 - val_loss: 2.4198 - val_accuracy: 0.1867

Epoch 00001: val_accuracy improved from -inf to 0.18673, saving model to /kaggle/working/Saved_CNN_Models/create_model_V1_1epochs/create_model_V1_model_5.h5


225/225 [==============================] - 6s 25ms/step - loss: 2.4378 - accuracy: 0.1895


In [15]:
def transfer_learning_model_resnet_50():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.resnet50.preprocess_input(inputs)
  base_model = tf.keras.applications.resnet50.ResNet50(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 150
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_resnet_50 = transfer_learning_model_resnet_50()

94781440/94765736 [==============================] - 0s 0us/step
175


In [16]:
transfer_model_resnet_50.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 48, 48, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 48, 48, 3)         0         
_________________________________________________________________
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 14343 

In [17]:
model_test(5, 50, transfer_learning_model_resnet_50, (48,48,), 'rgb')

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 30s - loss: 2.8674 - accuracy: 0.2513 - val_loss: 2.0466 - val_accuracy: 0.2955

Epoch 00001: val_accuracy improved from -inf to 0.29554, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_1.h5


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



Epoch 2/50
718/718 - 26s - loss: 1.9671 - accuracy: 0.3795 - val_loss: 1.8857 - val_accuracy: 0.3370

Epoch 00002: val_accuracy improved from 0.29554 to 0.33699, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_1.h5
Epoch 3/50
718/718 - 25s - loss: 1.4946 - accuracy: 0.5116 - val_loss: 1.7966 - val_accuracy: 0.4093

Epoch 00003: val_accuracy improved from 0.33699 to 0.40927, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_1.h5
Epoch 4/50
718/718 - 25s - loss: 1.0925 - accuracy: 0.6390 - val_loss: 1.8948 - val_accuracy: 0.4342

Epoch 00004: val_accuracy improved from 0.40927 to 0.43417, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_1.h5
Epoch 5/50
718/718 - 25s - loss: 0.8572 - accuracy: 0.7211 - val_loss: 1.7252 - val_accuracy: 0.4833

E

225/225 [==============================] - 12s 54ms/step - loss: 3.2472 - accuracy: 0.5198
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 29s - loss: 2.8466 - accuracy: 0.2523 - val_loss: 1.8720 - val_accuracy: 0.3384

Epoch 00001: val_accuracy improved from -inf to 0.33838, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_2.h5
Epoch 2/50
718/718 - 25s - loss: 1.9388 - accuracy: 0.3927 - val_loss: 1.7754 - val_accuracy: 0.3671

Epoch 00002: val_accuracy improved from 0.33838 to 0.36712, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_2.h5
Epoch 3/50
718/718 - 25s - loss: 1.4415 - accuracy: 0.5311 - val_loss: 1.7918 - val_accuracy: 0.4220

Epoch 00003: val_accuracy improved from 0.36712 to 0.42198, saving model to /kaggle/w

225/225 [==============================] - 12s 53ms/step - loss: 2.9069 - accuracy: 0.5029
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 30s - loss: 2.8706 - accuracy: 0.2551 - val_loss: 2.2823 - val_accuracy: 0.2823

Epoch 00001: val_accuracy improved from -inf to 0.28231, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_3.h5
Epoch 2/50
718/718 - 26s - loss: 1.9331 - accuracy: 0.3952 - val_loss: 1.8610 - val_accuracy: 0.3635

Epoch 00002: val_accuracy improved from 0.28231 to 0.36346, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_3.h5
Epoch 3/50
718/718 - 25s - loss: 1.4491 - accuracy: 0.5301 - val_loss: 1.7691 - val_accuracy: 0.4119

Epoch 00003: val_accuracy improved from 0.36346 to 0.41188, saving model to /kaggle/w

225/225 [==============================] - 12s 52ms/step - loss: 2.9696 - accuracy: 0.5276
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 29s - loss: 2.8566 - accuracy: 0.2542 - val_loss: 1.9983 - val_accuracy: 0.3004

Epoch 00001: val_accuracy improved from -inf to 0.30042, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_4.h5
Epoch 2/50
718/718 - 24s - loss: 1.9542 - accuracy: 0.3849 - val_loss: 1.9797 - val_accuracy: 0.3394

Epoch 00002: val_accuracy improved from 0.30042 to 0.33943, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_4.h5
Epoch 3/50
718/718 - 25s - loss: 1.4551 - accuracy: 0.5249 - val_loss: 1.6329 - val_accuracy: 0.4483

Epoch 00003: val_accuracy improved from 0.33943 to 0.44828, saving model to /kaggle/w

225/225 [==============================] - 13s 56ms/step - loss: 3.0380 - accuracy: 0.5103
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
175
Epoch 1/50
718/718 - 28s - loss: 2.8447 - accuracy: 0.2544 - val_loss: 2.0893 - val_accuracy: 0.3029

Epoch 00001: val_accuracy improved from -inf to 0.30291, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_5.h5
Epoch 2/50
718/718 - 25s - loss: 1.9448 - accuracy: 0.3924 - val_loss: 1.7908 - val_accuracy: 0.3813

Epoch 00002: val_accuracy improved from 0.30291 to 0.38129, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_50epochs/transfer_learning_model_resnet_50_model_5.h5
Epoch 3/50
718/718 - 25s - loss: 1.4772 - accuracy: 0.5193 - val_loss: 1.6549 - val_accuracy: 0.4386

Epoch 00003: val_accuracy improved from 0.38129 to 0.43860, saving model to /kaggle/w

225/225 [==============================] - 15s 68ms/step - loss: 3.0563 - accuracy: 0.5152


In [18]:
def transfer_learning_model_vgg_16():

  IMG_SHAPE = (48,48,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.vgg16.preprocess_input(inputs)
  base_model = tf.keras.applications.vgg16.VGG16(input_shape=(48,48,3,),
                                               include_top=False,
                                               weights='imagenet')
  print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 15
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_vgg_16 = transfer_learning_model_vgg_16()

58900480/58889256 [==============================] - 0s 0us/step
19


In [19]:
transfer_model_vgg_16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 48, 48, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 48, 48, 3)         0         
_________________________________________________________________
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 7)                 3591  

In [20]:
model_test(5, 50, transfer_learning_model_vgg_16, (48,48,), 'rgb')

Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 24s - loss: 3.1632 - accuracy: 0.1719 - val_loss: 2.0907 - val_accuracy: 0.1688

Epoch 00001: val_accuracy improved from -inf to 0.16876, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_1.h5
Epoch 2/50
718/718 - 23s - loss: 2.6155 - accuracy: 0.1687 - val_loss: 1.8601 - val_accuracy: 0.1688

Epoch 00002: val_accuracy did not improve from 0.16876
Epoch 3/50
718/718 - 23s - loss: 2.3907 - accuracy: 0.2313 - val_loss: 2.0714 - val_accuracy: 0.2966

Epoch 00003: val_accuracy improved from 0.16876 to 0.29659, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_1.h5
Epoch 4/50
718/718 - 22s - loss: 2.1871 - accuracy: 0.3475 - val_loss: 1.8172 - val_accuracy: 0.3406

Epoch 00004: val_accuracy improv

225/225 [==============================] - 12s 55ms/step - loss: 3.6880 - accuracy: 0.5836
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 24s - loss: 3.3147 - accuracy: 0.1676 - val_loss: 2.3150 - val_accuracy: 0.1684

Epoch 00001: val_accuracy improved from -inf to 0.16841, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_2.h5
Epoch 2/50
718/718 - 23s - loss: 2.5296 - accuracy: 0.2357 - val_loss: 1.9024 - val_accuracy: 0.3011

Epoch 00002: val_accuracy improved from 0.16841 to 0.30111, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_2.h5
Epoch 3/50
718/718 - 22s - loss: 2.2393 - accuracy: 0.3206 - val_loss: 1.7286 - val_accuracy: 0.3912

Epoch 00003: val_accuracy improved from 0.30111 to 0.39115, saving model to /kaggle/working/Saved_

225/225 [==============================] - 15s 66ms/step - loss: 2.6263 - accuracy: 0.5779
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 23s - loss: 3.2586 - accuracy: 0.1694 - val_loss: 2.3415 - val_accuracy: 0.1682

Epoch 00001: val_accuracy improved from -inf to 0.16823, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_3.h5
Epoch 2/50
718/718 - 22s - loss: 2.6743 - accuracy: 0.1682 - val_loss: 1.9979 - val_accuracy: 0.1682

Epoch 00002: val_accuracy did not improve from 0.16823
Epoch 3/50
718/718 - 23s - loss: 2.5273 - accuracy: 0.1683 - val_loss: 2.1239 - val_accuracy: 0.1682

Epoch 00003: val_accuracy did not improve from 0.16823
Epoch 4/50
718/718 - 22s - loss: 2.3779 - accuracy: 0.2009 - val_loss: 1.8349 - val_accuracy: 0.2948

Epoch 00004: val_accuracy improved from 0.16823 to 0.29485, saving model to /

225/225 [==============================] - 13s 57ms/step - loss: 2.5883 - accuracy: 0.5719
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 24s - loss: 3.6030 - accuracy: 0.1689 - val_loss: 2.0970 - val_accuracy: 0.1682

Epoch 00001: val_accuracy improved from -inf to 0.16823, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_4.h5
Epoch 2/50
718/718 - 22s - loss: 2.6779 - accuracy: 0.1684 - val_loss: 2.0636 - val_accuracy: 0.1682

Epoch 00002: val_accuracy did not improve from 0.16823
Epoch 3/50
718/718 - 22s - loss: 2.4265 - accuracy: 0.1992 - val_loss: 2.2766 - val_accuracy: 0.2346

Epoch 00003: val_accuracy improved from 0.16823 to 0.23459, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_4.h5
Epoch 4/50
718/718 - 22s - loss: 2.2156 - a

225/225 [==============================] - 12s 53ms/step - loss: 3.4375 - accuracy: 0.5890
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
19
Epoch 1/50
718/718 - 24s - loss: 3.4689 - accuracy: 0.1677 - val_loss: 2.2032 - val_accuracy: 0.1679

Epoch 00001: val_accuracy improved from -inf to 0.16792, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_5.h5
Epoch 2/50
718/718 - 22s - loss: 2.6871 - accuracy: 0.1712 - val_loss: 2.1573 - val_accuracy: 0.1705

Epoch 00002: val_accuracy improved from 0.16792 to 0.17053, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_50epochs/transfer_learning_model_vgg_16_model_5.h5
Epoch 3/50
718/718 - 23s - loss: 2.4799 - accuracy: 0.2384 - val_loss: 1.9163 - val_accuracy: 0.3078

Epoch 00003: val_accuracy improved from 0.17053 to 0.30779, saving model to /kaggle/working/Saved_

225/225 [==============================] - 14s 64ms/step - loss: 2.4355 - accuracy: 0.5745
